In [2]:
from joblib import load

std_scaler = load('./prediction_models/std_scaler.joblib')
linear_regression = load('./prediction_models/linear_regression.joblib')
random_forest = load('./prediction_models/random_forest.joblib')
neural_network = load('./prediction_models/neural_network.joblib')


In [3]:
import serial

arduino = serial.Serial(port='/dev/cu.usbserial-1430', baudrate=115200, timeout=.1)


In [45]:
import numpy as np

def predict(data):
    data = np.fromstring(data, dtype=float, sep=',').reshape(1, -1)
    scaled_data = std_scaler.transform(data)
    return (linear_regression.predict(scaled_data), random_forest.predict(data), neural_network.predict(scaled_data))


In [86]:
import numpy as np

mean_count = 3
lr_prediction = 0
rf_prediction = 0
nn_prediction = 0

# Continuous prediction
while True:
    # Send get command to arduino
    command = "get:3,max"
    arduino.write(bytes(command, 'utf-8'))
    while True:
        # Wait until arduino sends data
        if arduino.in_waiting > 0:
            line = arduino.readline()
            predictions = predict(line.decode())
            break
    
    if lr_prediction == 0:
        lr_prediction = predictions[0]
    else:
        lr_prediction = 0.9 * lr_prediction + 0.1 * predictions[0]
        
    if rf_prediction == 0:
        rf_prediction = predictions[1]
    else:
        rf_prediction = 0.9 * rf_prediction + 0.1 * predictions[1]
        
    if nn_prediction == 0:
        nn_prediction = predictions[2]
    else:
        nn_prediction = 0.9 * nn_prediction + 0.1 * predictions[2]
        
    print("LR: " + str(lr_prediction) + " RF: " + str(rf_prediction) + " NN: " + str(nn_prediction))

Random forest:  [5.80128555]
Linear regression:  [7.43924941]
Neural network:  [8.0945189]
